## Descripción del proyecto

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Objetivo del proyecto

- Predecir el precio de un coche
- Compara Moelos diferentes: Regrecion lineal, Arboles de decición, bosque Aleatario y Potenciacion Gradiante (lightGBM).
- Implementar Manual Mente un Regrecion Lineal con desenso por gradiante.

### Consideraciones 
- Regrecion lienal sirve para prueba de cordura 
- Protenciacon del gradiete debe fucionar meejor que la regreción lineal, si no algo , esta mal.
- LightGBM y GatBoost Maneja Categóricas; XGBoost Requiere OHE. 
- Usa %%time para medir tiempos en Jupyter.

## Preparación de datos

In [33]:
# Importación de librerías necesarias
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import display
from datetime import date
import time
from time import perf_counter

# Modelos de Gradient Boosting
import lightgbm as lgb
from lightgbm import LGBMRegressor
import xgboost as xgb

# Modelos de sklearn
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyClassifier

# Herramientas de evaluación y preprocesamiento
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold, cross_validate, train_test_split
from sklearn.metrics import mean_squared_error, r2_score , root_mean_squared_error, make_scorer
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.preprocessing import MaxAbsScaler, OrdinalEncoder, StandardScaler, OneHotEncoder

from sklearn.compose import ColumnTransformer

# Semilla para reproducibilidad de resultados
RANDOM_STATE=12345

In [34]:
# Carga del dataset desde archivo local o servidor
BASE_DIR = Path.cwd()

try: 
    DIR = BASE_DIR / 'car_data.csv'
    df = pd.read_csv(DIR)
    print(DIR)
except:
    # Ruta alternativa para ejecución en servidor de TripleTen
    df = pd.read_csv('/datasets/car_data.csv')

\\puebla\Programacion\Data Science\Tripleten ejercicios\Proyectos\Sprint 15\car_data.csv


In [35]:
# Revisión general del dataset: tipos de datos, nulos y tamaño
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [36]:
# Vista previa de las primeras filas para entender la estructura de los datos
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [37]:
# Inspección de un registro específico para verificar valores atípicos (precio=0, power=0)
df.loc[354364]

DateCrawled          21/03/2016 09:50
Price                               0
VehicleType                       NaN
RegistrationYear                 2005
Gearbox                        manual
Power                               0
Model                            colt
Mileage                        150000
RegistrationMonth                   7
FuelType                       petrol
Brand                      mitsubishi
NotRepaired                       yes
DateCreated          21/03/2016 00:00
NumberOfPictures                    0
PostalCode                       2694
LastSeen             21/03/2016 10:42
Name: 354364, dtype: object

In [38]:
# Conversión de columnas de fecha de texto (object) a formato datetime
# Esto permite operaciones temporales y evita errores al entrenar modelos
date_cols = ['DateCrawled', 'DateCreated', 'LastSeen']

for i in date_cols:
    df[i] = pd.to_datetime(df[i], format='%d/%m/%Y %H:%M')

In [39]:
# Verificación de la conversión de tipos de datos (las fechas ahora son datetime64)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        354369 non-null  datetime64[ns]
 1   Price              354369 non-null  int64         
 2   VehicleType        316879 non-null  object        
 3   RegistrationYear   354369 non-null  int64         
 4   Gearbox            334536 non-null  object        
 5   Power              354369 non-null  int64         
 6   Model              334664 non-null  object        
 7   Mileage            354369 non-null  int64         
 8   RegistrationMonth  354369 non-null  int64         
 9   FuelType           321474 non-null  object        
 10  Brand              354369 non-null  object        
 11  NotRepaired        283215 non-null  object        
 12  DateCreated        354369 non-null  datetime64[ns]
 13  NumberOfPictures   354369 non-null  int64   

In [40]:
# Estadísticas descriptivas: identificamos valores atípicos
# - Price min=0 (autos sin precio válido)

# - Power min=0 y max=20000 (valores imposibles)

# - RegistrationYear max=9999 (año inválido)

# - RegistrationMonth min=0 (mes inválido)

df.describe()

,DateCrawled,Price,RegistrationYear,Power,Mileage,RegistrationMonth,DateCreated,NumberOfPictures,PostalCode,LastSeen
count,354369,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369,354369.0,354369.000000,354369
mean,2016-03-21 12:57:41.165057280,4416.656776,2004.234448,110.094337,128211.172535,5.714645,2016-03-20 19:12:07.753274112,0.0,50508.689087,2016-03-29 23:50:30.593703680
min,2016-03-05 14:06:00,0.000000,1000.000000,0.000000,5000.000000,0.000000,2014-03-10 00:00:00,0.0,1067.000000,2016-03-05 14:15:00
25%,2016-03-13 11:52:00,1050.000000,1999.000000,69.000000,125000.000000,3.000000,2016-03-13 00:00:00,0.0,30165.000000,2016-03-23 02:50:00
50%,2016-03-21 17:50:00,2700.000000,2003.000000,105.000000,150000.000000,6.000000,2016-03-21 00:00:00,0.0,49413.000000,2016-04-03 15:15:00
75%,2016-03-29 14:37:00,6400.000000,2008.000000,143.000000,150000.000000,9.000000,2016-03-29 00:00:00,0.0,71083.000000,2016-04-06 10:15:00
max,2016-04-07 14:36:00,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,2016-04-07 00:00:00,0.0,99998.000000,2016-04-07 14:58:00
std,NaN,4514.158514,90.227958,189.850405,37905.341530,3.726421,NaN,0.0,25783.096248,NaN


### Anotaciones

- El poder de los veiculos tenemos desde los 0 CV hasta los 
- En el precio tambien encontramos valores en 0 
- Tenemos Año de registro como 9999 lo cual no es posible
- Contamos con 354,369 registros en total
- Hay mes de registro 0 

In [41]:
# Análisis de valores nulos: NotRepaired tiene ~20% de nulos, VehicleType ~10%

# Estos porcentajes son significativos y deben considerarse en la limpieza
nulos = df.isna().sum()
porcentaje = df.isna().mean().mul(100)
resumen = pd.DataFrame({"null": nulos, "%": porcentaje}).sort_values("null", ascending=False)
resumen

,null,%
NotRepaired,71154,20.079070
VehicleType,37490,10.579368
FuelType,32895,9.282697
Gearbox,19833,5.596709
Model,19705,5.560588
Price,0,0.000000
RegistrationYear,0,0.000000
DateCrawled,0,0.000000
Mileage,0,0.000000
Power,0,0.000000


In [42]:
# Exploración de filas donde VehicleType es nulo

# Muchos de estos registros también tienen otros campos nulos (FuelType, NotRepaired)
null_vehi = df[df['VehicleType'].isnull()]
null_vehi

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:00,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24,0,70435,2016-04-07 03:16:00
16,2016-04-01 12:46:00,300,NaN,2016,NaN,60,polo,150000,0,petrol,volkswagen,NaN,2016-04-01,0,38871,2016-04-01 12:46:00
22,2016-03-23 14:52:00,2900,NaN,2018,manual,90,meriva,150000,5,petrol,opel,no,2016-03-23,0,49716,2016-03-31 01:16:00
26,2016-03-10 19:38:00,5555,NaN,2017,manual,125,c4,125000,4,NaN,citroen,no,2016-03-10,0,31139,2016-03-16 09:16:00
31,2016-03-29 16:57:00,899,NaN,2016,manual,60,clio,150000,6,petrol,renault,NaN,2016-03-29,0,37075,2016-03-29 17:43:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354346,2016-03-07 17:06:00,2600,NaN,2005,auto,0,c_klasse,150000,9,NaN,mercedes_benz,NaN,2016-03-07,0,61169,2016-03-08 21:28:00
354351,2016-03-11 23:40:00,1900,NaN,2000,manual,110,NaN,150000,7,NaN,volkswagen,no,2016-03-11,0,87700,2016-03-12 14:16:00
354361,2016-03-09 13:37:00,5250,NaN,2016,auto,150,159,150000,12,NaN,alfa_romeo,no,2016-03-09,0,51371,2016-03-13 01:44:00
354364,2016-03-21 09:50:00,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes,2016-03-21,0,2694,2016-03-21 10:42:00


In [43]:
# Filtrado de outliers usando rangos razonables para cada variable numérica

# Se eliminan registros con valores imposibles o extremos que distorsionan el modelo

# Filtro de Kilometraje: entre percentil 1 y 99
low_m = df["Mileage"].quantile(0.01)
high_m = df["Mileage"].quantile(0.99)

# Filtro de precio: mínimo €100 (excluye autos "gratis") y máximo €50,000
low_p = 100
high_p = 50000

# Filtro de caballos de fuerza: entre 16 y 500 CV (valores realistas)
low_pow = 16
high_pow = 500

# Filtro de año de registro: entre 1950 y el año actual
low_y = 1950
high_y = date.today().year

# Aplicación de todos los filtros simultáneamente
df_filtrado_x = df[
    (df["Mileage"].between(low_m, high_m)) &
    (df["Price"].between(low_p, high_p)) &
    (df["Power"].between(low_pow, high_pow))&
    (df["RegistrationYear"].between(low_y, high_y))
]

In [44]:
# Verificación del dataset filtrado: pasamos de 354,369 a ~305,203 registros

# Se eliminaron ~49,000 filas con valores atípicos
df_filtrado_x.info()

<class 'pandas.core.frame.DataFrame'>
Index: 305203 entries, 1 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        305203 non-null  datetime64[ns]
 1   Price              305203 non-null  int64         
 2   VehicleType        284200 non-null  object        
 3   RegistrationYear   305203 non-null  int64         
 4   Gearbox            299281 non-null  object        
 5   Power              305203 non-null  int64         
 6   Model              293099 non-null  object        
 7   Mileage            305203 non-null  int64         
 8   RegistrationMonth  305203 non-null  int64         
 9   FuelType           285779 non-null  object        
 10  Brand              305203 non-null  object        
 11  NotRepaired        258737 non-null  object        
 12  DateCreated        305203 non-null  datetime64[ns]
 13  NumberOfPictures   305203 non-null  int64        

In [45]:
# Revisión de autos con muy baja potencia (<=20 CV)

# Incluye autos clásicos (Fiat 500 de 1954) y posibles errores de captura
df_filtrado_x[df_filtrado_x['Power']<=20].sort_values(by='RegistrationYear', ascending=True)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
37133,2016-03-12 18:47:00,7500,small,1954,manual,16,500,125000,7,petrol,fiat,no,2016-03-12,0,82327,2016-03-14 11:17:00
341072,2016-04-04 22:55:00,9990,sedan,1954,manual,20,NaN,30000,6,petrol,sonstige_autos,no,2016-04-04,0,1744,2016-04-07 01:46:00
101154,2016-03-31 21:56:00,14800,sedan,1956,manual,20,other,70000,7,petrol,renault,no,2016-03-31,0,91217,2016-04-04 16:45:00
103392,2016-03-25 18:55:00,7600,small,1957,manual,19,NaN,30000,3,petrol,sonstige_autos,no,2016-03-25,0,13583,2016-04-07 01:18:00
320159,2016-03-22 20:49:00,6650,small,1959,NaN,19,500,40000,4,NaN,fiat,no,2016-03-22,0,22848,2016-03-24 05:16:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114041,2016-04-03 19:47:00,2000,NaN,2017,manual,16,micra,150000,0,NaN,nissan,NaN,2016-04-03,0,54295,2016-04-05 20:44:00
299885,2016-03-31 20:40:00,2600,NaN,2017,NaN,17,NaN,5000,7,petrol,sonstige_autos,NaN,2016-03-31,0,92431,2016-03-31 20:40:00
170640,2016-03-12 18:37:00,950,NaN,2017,manual,16,astra,30000,0,NaN,opel,NaN,2016-03-12,0,47229,2016-03-15 05:15:00
232344,2016-03-13 13:37:00,800,small,2017,manual,16,a3,30000,0,petrol,audi,yes,2016-03-13,0,47053,2016-03-28 12:46:00


In [46]:
# NumberOfPictures tiene valor 0 en todos los registros: no aporta información

# Se descartará como feature para el modelo
df_filtrado_x['NumberOfPictures'].value_counts()

NumberOfPictures
0    305203
Name: count, dtype: int64

In [47]:
# Nulos restantes tras el filtrado de outliers

# NotRepaired sigue con ~46k nulos; se eliminarán con dropna() más adelante
display(df_filtrado_x.isna().sum())

DateCrawled              0
Price                    0
VehicleType          21003
RegistrationYear         0
Gearbox               5922
Power                    0
Model                12104
Mileage                  0
RegistrationMonth        0
FuelType             19424
Brand                    0
NotRepaired          46466
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

In [48]:
# Verificación de duplicados: solo 254 filas duplicadas (~0.08%), cantidad insignificante
print(df_filtrado_x.duplicated().sum())

254


### Preparación de datos
- El dataset original contenía **354,369 registros** con valores atípicos significativos: precios en 0, potencia de 0 a 20,000 CV y años de registro hasta 9999.
- Tras aplicar filtros por rangos razonables (precio €100-€50,000, potencia 16-500 CV, año 1950-actual) y eliminar nulos, quedaron **232,499 registros limpios** (~65% del original).
- La columna `NumberOfPictures` fue descartada por contener únicamente ceros (sin valor predictivo).
- Se aplicó **One-Hot Encoding** a 6 variables categóricas, generando 306 features, y **StandardScaler** solo a las numéricas reales para evitar alterar las dummies.


## Entrenamiento del modelo 

In [49]:
# Selección de features relevantes para el modelo

# Se excluyen: DateCrawled, DateCreated, LastSeen (fechas del anuncio, no del auto)

# Se excluye: NumberOfPictures (siempre 0), RegistrationMonth (poco predictivo)
col_features = ['Price',
            'VehicleType', 
            'RegistrationYear', 
            'Gearbox', 
            'Power', 
            'Model', 
            'Mileage', 
            'FuelType', 
            'Brand', 
            'NotRepaired', 
            'PostalCode'] 

df_clean =  df_filtrado_x[col_features]

# Eliminación de filas con valores nulos en las columnas seleccionadas

# Pasamos de ~305k a ~232k registros (perdemos ~24% por nulos)
df_clean = df_clean.dropna()

df_clean

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,FuelType,Brand,NotRepaired,PostalCode
3,1500,small,2001,manual,75,golf,150000,petrol,volkswagen,no,91074
4,3600,small,2008,manual,69,fabia,90000,gasoline,skoda,no,60437
5,650,sedan,1995,manual,102,3er,150000,petrol,bmw,yes,33775
6,2200,convertible,2004,manual,109,2_reihe,150000,petrol,peugeot,no,67112
10,2000,sedan,2004,manual,105,3_reihe,150000,petrol,mazda,no,96224
...,...,...,...,...,...,...,...,...,...,...,...
354358,1490,small,1998,manual,50,lupo,150000,petrol,volkswagen,no,48653
354359,7900,sedan,2010,manual,140,golf,150000,gasoline,volkswagen,no,75223
354362,3200,sedan,2004,manual,225,leon,150000,petrol,seat,yes,96465
354366,1199,convertible,2000,auto,101,fortwo,125000,petrol,smart,no,26135


In [50]:
from sklearn.preprocessing import StandardScaler

# Codificación One-Hot de variables categóricas

# drop_first=True evita multicolinealidad (la primera categoría se infiere)

# Esto es necesario para modelos lineales y XGBoost

categorical = [ 'VehicleType',  
            'Gearbox',  
            'Model', 
            'FuelType', 
            'Brand', 
            'NotRepaired',
            ] 

data_ohe = pd.get_dummies(df_clean, columns=categorical, drop_first=True)

In [51]:
# Resultado del OHE: de 11 columnas pasamos a 306 (muchas dummies por Model y Brand)
data_ohe

,Price,RegistrationYear,Power,Mileage,PostalCode,VehicleType_convertible,VehicleType_coupe,VehicleType_other,VehicleType_sedan,VehicleType_small,...,Brand_seat,Brand_skoda,Brand_smart,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_yes
3,1500,2001,75,150000,91074,False,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
4,3600,2008,69,90000,60437,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
5,650,1995,102,150000,33775,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,True
6,2200,2004,109,150000,67112,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10,2000,2004,105,150000,96224,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354358,1490,1998,50,150000,48653,False,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
354359,7900,2010,140,150000,75223,False,False,False,True,False,...,False,False,False,False,False,False,False,True,False,False
354362,3200,2004,225,150000,96465,False,False,False,True,False,...,True,False,False,False,False,False,False,False,False,True
354366,1199,2000,101,125000,26135,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False


In [52]:
# Separación de features (X) y variable objetivo (y = Price)
target_col = "Price"  

X = data_ohe.drop(columns=[target_col])
y = data_ohe[target_col]

# División en conjuntos de entrenamiento (75%) y validación (25%)
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.25, random_state=RANDOM_STATE
)

print(
    '\nX_train:', X_train.size,
    '\nX_valid:', X_valid.size,
    '\ny_train:', y_train.size,
    '\ny_valid:', y_valid.size
    )

# Diccionario para almacenar tiempos de entrenamiento de cada modelo
tiempos = {}
# Diccionario para almacenar tiempos de predicción de cada modelo
tiempos_pred = {}


X_train: 53184070 
X_valid: 17728125 
y_train: 174374 
y_valid: 58125


In [53]:
from sklearn.preprocessing import StandardScaler

# Escalado de variables numéricas con StandardScaler (media=0, desviación=1)

# Solo escalamos las numéricas reales, NO las dummies (bool/uint8)

# Esto es importante para modelos lineales y descenso por gradiente


# Detectar columnas numéricas evitando booleanas
num_cols = X_train.select_dtypes(include=["int64","float64"]).columns
dummy_cols = X_train.select_dtypes(include=["uint8","bool"]).columns
scale_cols = [c for c in num_cols if c not in dummy_cols]

scaler = StandardScaler()

# Creamos copias para no modificar los datos originales
X_train_s = X_train.copy()
X_valid_s = X_valid.copy()

# fit_transform en train, solo transform en valid (evita data leakage)
X_train_s[scale_cols] = scaler.fit_transform(X_train[scale_cols])
X_valid_s[scale_cols] = scaler.transform(X_valid[scale_cols])

In [54]:
X_valid_s.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58125 entries, 295662 to 50415
Columns: 305 entries, RegistrationYear to NotRepaired_yes
dtypes: bool(301), float64(4)
memory usage: 18.9 MB


In [55]:
import numpy as np

# Conversión a numpy float64 para asegurar compatibilidad numérica

# Necesario para la implementación manual de regresión lineal con descenso por gradiente
X_num = np.asarray(X, dtype=np.float64)
y_num = np.asarray(y, dtype=np.float64).ravel()

print("X_num:", X_num.shape, X_num.dtype)
print("y_num:", y_num.shape, y_num.dtype)

X_num: (232499, 305) float64
y_num: (232499,) float64


In [ ]:
# ===== MODELO 1: REGRESIÓN LINEAL (BASELINE / PRUEBA DE CORDURA) =====


# Sirve como referencia mínima: si ningún modelo supera esto, hay un problema
t0 = perf_counter()

model_linear = LinearRegression()
model_linear.fit(X_train_s, y_train)

t_pred = perf_counter()
pediction_linear= model_linear.predict(X_valid_s)

tiempos_pred["LinearRegression"] = perf_counter() - t_pred
tiempos["LinearRegression"] = perf_counter() - t0

# Métricas: R² indica qué % de la varianza explica el modelo
# RMSE está en las mismas unidades que el precio (euros)
r2_linear =  r2_score(y_valid, pediction_linear)
mse_linear = mean_squared_error(y_valid, pediction_linear)
rmse_linear = root_mean_squared_error(y_valid, pediction_linear)

print('R2:', r2_linear)
print('MSE:', mse_linear)
print('RMSE:', rmse_linear)

R2: 0.7169492438156099
MSE: 6291182.7350181965
RMSE: 2508.2230233809346


In [ ]:
# ===== MODELO 2: ÁRBOL DE DECISIÓN CON BÚSQUEDA DE HIPERPARÁMETROS =====


# Se usa GridSearchCV para encontrar la mejor combinación de max_depth y min_samples_split

# cv=2 para reducir tiempo de búsqueda dado el tamaño del dataset

t0 = perf_counter()

model_tree = DecisionTreeRegressor(random_state=RANDOM_STATE)
model_tree.fit(X_train_s, y_train)

# Cuadrícula de hiperparámetros a explorar
param_grid = {
    'max_depth': [10, 100, 150],
    'min_samples_split': [2,5],
}

# Scorer personalizado basado en RMSE (negativo porque sklearn maximiza)
rmse_score = make_scorer(mean_squared_error, greater_is_better=False, squared=False)
grid_serch = GridSearchCV(estimator=model_tree, param_grid=param_grid, scoring=rmse_score, cv=2, n_jobs=-1)

grid_serch.fit(X_train_s, y_train)

# Se usa el mejor modelo encontrado para predecir
best_model = grid_serch.best_estimator_

t_pred = perf_counter()
pediction_tree= best_model.predict(X_valid_s)

tiempos_pred["DecisionTree"] = perf_counter() - t_pred
tiempos["DecisionTree"] = perf_counter() - t0

r2_tree = r2_score(y_valid, pediction_tree)
mse_tree = mean_squared_error(y_valid, pediction_tree)
rmse_tree = root_mean_squared_error(y_valid ,pediction_tree)


print(f'Mejor combinación: {grid_serch.best_params_}')
print('R2:', r2_tree)
print('MSE:', mse_tree)
print('RMSE:', rmse_tree)

\\puebla\Programacion\Data Science\Tripleten ejercicios\.venv\Lib\site-packages\sklearn\model_selection\_search.py:1137: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(


Mejor combinación: {'max_depth': 10, 'min_samples_split': 2}
R2: 0.8295444511520479
MSE: 3788603.2189283613
RMSE: 1946.4334612126768


In [58]:
# ===== MODELO 3: BOSQUE ALEATORIO (RANDOM FOREST) =====


# Ensemble de 100 árboles que reduce el sobreajuste del árbol individual

# n_jobs=-1 usa todos los núcleos del CPU para paralelizar
t0 = perf_counter()

model_forest = RandomForestRegressor(n_estimators=100 , n_jobs=-1, random_state=RANDOM_STATE)
model_forest.fit(X_train_s, y_train)

t_pred = perf_counter()
pediction_forest= model_forest.predict(X_valid_s)

tiempos_pred["RandomForest"] = perf_counter() - t_pred
tiempos["RandomForest"] = perf_counter() - t0

r2_forest = r2_score(y_valid, pediction_forest)
mse_forest = mean_squared_error(y_valid, pediction_forest)
rmse_forest = root_mean_squared_error(y_valid, pediction_forest)

print('R2:', r2_forest)
print('MSE:', mse_forest)
print('RMSE:', rmse_forest)

R2: 0.8959532139482573
MSE: 2312579.3863501935
RMSE: 1520.7167344216982


In [59]:
# ===== MODELO 4: LightGBM con early stopping (API sklearn) =====


# Potenciación por gradiente: construye árboles secuencialmente, cada uno corrige errores del anterior

# Early stopping detiene el entrenamiento si no mejora en 200 rondas (evita sobreajuste)
t0 = perf_counter()

model_light_GBM = LGBMRegressor(
    objective="regression",
    n_estimators=10_000,       # alto número + early stopping para encontrar el óptimo
    learning_rate=0.05,        # paso de aprendizaje moderado
    num_leaves=31,             # complejidad de cada árbol
    subsample=0.8,             # usa 80% de datos por árbol (regularización)
    colsample_bytree=0.8,     # usa 80% de features por árbol (regularización)
    random_state=42,
    n_jobs=-1
)

model_light_GBM.fit(
    X_train_s, y_train,
    eval_set=[(X_valid_s, y_valid)],  # monitorea el error en validación
    eval_metric="rmse",
    callbacks=[
        lgb.early_stopping(stopping_rounds=200),  # para si no mejora en 200 rondas
        lgb.log_evaluation(period=200),            # imprime cada 200 rondas
    ]
)

t_pred = perf_counter()
pred = model_light_GBM.predict(X_valid_s)

tiempos_pred["LGBMR"] = perf_counter() - t_pred
tiempos["LGBMR"] = perf_counter() - t0

r2_lgbmr_early = r2_score(y_valid, pred)
mse_lgbmr_early = mean_squared_error(y_valid, pred)
rmse_lgbmr_early = root_mean_squared_error(y_valid, pred)

print('R2:', r2_lgbmr_early)
print('MSE:', mse_lgbmr_early)
print('RMSE:', rmse_lgbmr_early)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002910 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1122
[LightGBM] [Info] Number of data points in the train set: 174374, number of used features: 283
[LightGBM] [Info] Start training from score 5311.905949
Training until validation scores don't improve for 200 rounds
[200]	valid_0's rmse: 1623.91	valid_0's l2: 2.63707e+06
[400]	valid_0's rmse: 1569.81	valid_0's l2: 2.46429e+06
[600]	valid_0's rmse: 1545.4	valid_0's l2: 2.38825e+06
[800]	valid_0's rmse: 1531.6	valid_0's l2: 2.34579e+06
[1000]	valid_0's rmse: 1521.4	valid_0's l2: 2.31467e+06
[1200]	valid_0's rmse: 1512.67	valid_0's l2: 2.28817e+06
[1400]	valid_0's rmse: 1506.41	valid_0's l2: 2.26926e+06
[1600]	valid_0's rmse: 1501.31	valid_0's l2: 2.25394e+06
[1800]	valid_0's rmse: 1495.94	valid_0's l2: 2.23785e+06
[2000]	valid_0's rmse

In [60]:
# ===== MODELO 5: LightGBM con API nativa (lgb.train) =====


# Diferencia con LGBMR: usa la API nativa de LightGBM (más flexible)

# Solo 100 rondas de boosting (sin early stopping) para comparar velocidad
t0 = perf_counter()

lgb_train = lgb.Dataset(X_train_s, y_train)

params = {
    'objective': "regression",
    'n_estimators': 10_000,       
    'learning_rate': 0.05,
    'num_leaves': 31,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'random_state': 42,
    'n_jobs': -1
}
gbm = lgb.train(params, lgb_train, num_boost_round=100)

t_pred = perf_counter()
preds_lgb = gbm.predict(X_valid_s)

tiempos_pred["LGB"] = perf_counter() - t_pred
tiempos["LGB"] = perf_counter() - t0

r2_lgb = r2_score(y_valid, preds_lgb)
mse_lgb = mean_squared_error(y_valid, preds_lgb)
rmse_lgn = root_mean_squared_error(y_valid, preds_lgb)

print('R2:', r2_lgb)
print('MSE:', mse_lgb)
print('RMSE:', rmse_lgn)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003237 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1122
[LightGBM] [Info] Number of data points in the train set: 174374, number of used features: 283
[LightGBM] [Info] Start training from score 5311.905949
R2: 0.9027376570929682
MSE: 2161786.036937935
RMSE: 1470.3013422213608


In [61]:
# ===== MODELO 6: REGRESIÓN LINEAL MANUAL CON DESCENSO POR GRADIENTE =====


# Implementación desde cero para demostrar cómo funciona el algoritmo internamente

# Usa Batch Gradient Descent: calcula el gradiente con TODOS los datos en cada época

class GDLinearRegression:
    def __init__(self, lr=0.1, epochs=200):
        # Learning rate (tamaño del paso):
        #   - Si es muy grande: los pesos rebotan y no convergen
        #   - Si es muy chico: aprende muy lento
        # IMPORTANTE: requiere datos escalados para funcionar correctamente
        self.lr = lr
        
        # Épocas: número de pasadas completas sobre el dataset
        self.epochs = epochs
        
        # Pesos (w) e intercepto (b) se inicializan en fit()
        self.w = None
        self.b = None
        
    def fit(self, X, y):
        # Convertimos a numpy float64 para evitar errores de tipo
        Xn = X.to_numpy(dtype=np.float64)          # (n, p)
        yn = y.to_numpy(dtype=np.float64).ravel()  # (n,)
        
        # n = número de muestras, p = número de features
        n, p = Xn.shape
        
        # Inicialización de pesos en ceros
        self.w = np.zeros(p, dtype=np.float64)
        self.b = 0.0
        
        # Bucle de entrenamiento (Batch Gradient Descent)
        for epoch in range(self.epochs):
            
            # Paso 1: Predicción actual -> y_hat = Xw + b
            y_hat = Xn @ self.w + self.b
            
            # Paso 2: Calcular el error (residuo)
            e = y_hat - yn
            
            # Paso 3: Calcular gradientes de MSE
            # Derivada parcial respecto a w: grad_w = (2/n) * X^T * e
            grad_w = (2/n) * (Xn.T @ e)
            
            # Derivada parcial respecto a b: grad_b = (2/n) * sum(e)
            grad_b = (2/n) * np.sum(e)
            
            # Paso 4: Actualizar parámetros en dirección contraria al gradiente
            self.w -= self.lr * grad_w
            self.b -= self.lr * grad_b
        
        return self
    
    def predict(self, X):
        Xn = X.to_numpy(dtype=np.float64)
        return Xn @ self.w + self.b

In [62]:
# Entrenamiento y evaluación de la regresión lineal manual
# Usa datos escalados (X_train_s) para que el gradiente converja correctamente

t0 = perf_counter()

model_linear_GD = GDLinearRegression(lr=0.1, epochs=200)
model_linear_GD.fit(X_train_s, y_train)

t_pred = perf_counter()
prediction_linear_GD = model_linear_GD.predict(X_valid_s)

tiempos_pred["LinearRegression_GD"] = perf_counter() - t_pred
tiempos["LinearRegression_GD"] = perf_counter() - t0

r2_linear_GD  = r2_score(y_valid, prediction_linear_GD)
mse_linear_GD = mean_squared_error(y_valid, prediction_linear_GD)
rmse_linear_GD = root_mean_squared_error(y_valid, prediction_linear_GD)

print('R2:', r2_linear_GD)
print('MSE:', mse_linear_GD)
print('RMSE:', rmse_linear_GD)

R2: 0.6814762280833605
MSE: 7079618.092485945
RMSE: 2660.755173345707


In [ ]:
# ===== MODELO 7: XGBoost =====


# Otro algoritmo de potenciación por gradiente, competidor de LightGBM

# tree_method="hist" usa histogramas para acelerar el entrenamiento

# XGBoost requiere OHE (ya aplicado), a diferencia de LightGBM que maneja categóricas

t0 = perf_counter()

reg = xgb.XGBRegressor(tree_method="hist")
reg.fit(X_train_s, y_train)

t_pred = perf_counter()
xgb_reg = reg.predict(X_valid_s)

tiempos_pred["XGBoost"] = perf_counter() - t_pred
tiempos["XGBoost"] = perf_counter() - t0

r2_XGBoost  = r2_score(y_valid, xgb_reg)
mse_r2_XGBoost = mean_squared_error(y_valid, xgb_reg)
rmse_r2_XGBoost = root_mean_squared_error(y_valid, xgb_reg)

print('R2:', r2_XGBoost)
print('MSE:', mse_r2_XGBoost)
print('RMSE:', rmse_r2_XGBoost)

R2: 0.8874062895774841
MSE: 2502546.75
RMSE: 1581.9439697265625


In [69]:
# ===== TABLA COMPARATIVA DE TODOS LOS MODELOS =====


# Resumen con R², MSE, RMSE, tiempo total y tiempo de predicción ordenado por mejor RMSE
tabla_resultados = pd.DataFrame([
    {"Modelo": "LinearRegression",  "R2": r2_linear,  "MSE": mse_linear,  "RMSE": rmse_linear, "Tiempo_s": tiempos["LinearRegression"], "Pred_ms": tiempos_pred["LinearRegression"] * 1000},
    {"Modelo": "DecisionTree",  "R2": r2_tree,    "MSE": mse_tree,    "RMSE": rmse_tree, "Tiempo_s": tiempos["DecisionTree"], "Pred_ms": tiempos_pred["DecisionTree"] * 1000},
    {"Modelo": "RandomForest",  "R2": r2_forest,  "MSE": mse_forest,  "RMSE": rmse_forest, "Tiempo_s": tiempos["RandomForest"], "Pred_ms": tiempos_pred["RandomForest"] * 1000},
    {"Modelo": "LGBMR",  "R2": r2_lgbmr_early,  "MSE": mse_lgbmr_early,  "RMSE": rmse_lgbmr_early, "Tiempo_s": tiempos["LGBMR"], "Pred_ms": tiempos_pred["LGBMR"] * 1000},
    {"Modelo": "LGB",  "R2": r2_lgb,  "MSE": mse_lgb,  "RMSE": rmse_lgn, "Tiempo_s": tiempos["LGB"], "Pred_ms": tiempos_pred["LGB"] * 1000},
    {"Modelo": "LinearRegression_GD",  "R2": r2_linear_GD,  "MSE": mse_linear_GD,  "RMSE": rmse_linear_GD, "Tiempo_s": tiempos["LinearRegression_GD"], "Pred_ms": tiempos_pred["LinearRegression_GD"] * 1000},
    {"Modelo": "XGBoost",  "R2": r2_XGBoost,  "MSE": mse_r2_XGBoost,  "RMSE": rmse_r2_XGBoost, "Tiempo_s": tiempos["XGBoost"], "Pred_ms": tiempos_pred["XGBoost"] * 1000},    
]).set_index("Modelo")

tabla_resultados.sort_values("RMSE", ascending=True).style.format({
    "R2": "{:.4f}",
    "MSE": "{:,.0f}",
    "RMSE": "{:,.2f}",
    "Tiempo_s": "{:.3f}",
    "Pred_ms": "{:.2f}"
})

,R2,MSE,RMSE,Tiempo_s,Pred_ms
Modelo,,,,,
LGB,0.9027,"2,161,786","1,470.30",18.566,1354.08
LGBMR,0.9027,"2,162,785","1,470.64",13.322,870.78
RandomForest,0.8960,"2,312,579","1,520.72",34.553,336.47
XGBoost,0.8874,"2,502,547","1,581.94",1.246,94.11
DecisionTree,0.8295,"3,788,603","1,946.43",36.059,45.07
LinearRegression,0.7169,"6,291,183","2,508.22",2.266,97.76
LinearRegression_GD,0.6815,"7,079,618","2,660.76",6.084,31.70


## Conclusión

Con base en el **objetivo del proyecto** (predecir el precio de un coche), se compararon modelos de **Regresión Lineal**, **Árbol de Decisión**, **Bosque Aleatorio** y **Potenciación por Gradiente (LightGBM y XGBoost)**, además de implementar manualmente una **Regresión Lineal con Descenso por Gradiente**.



- **Prueba de cordura (Regresión Lineal):** `LinearRegression` funciona como baseline sólido (**RMSE = €2,508.22**, **R² = 0.7169**). Sirve para validar que el pipeline y las métricas están bien configurados.

- **Implementación manual (Descenso por Gradiente):** `LinearRegression_GD` quedó por debajo de la regresión lineal estándar (**RMSE = €2,660.76**, **R² = 0.6815**) y fue más lenta (6.32s vs 2.42s). Esto indica que los hiperparámetros (learning rate, epochs, early stopping) no convergen al óptimo esperado.

- **Modelos de árboles:**  
  - `DecisionTree` generaliza peor (**RMSE = €1,946.43**, **R² = 0.8295**, **36.24s**), indicando sobreajuste.
  - `RandomForest` mejora significativamente (**RMSE = €1,520.72**, **R² = 0.8960**, **34.28s**), pero con alto costo computacional.

- **Potenciación por gradiente (supera expectativas):** 
  - `LGB` logra **RMSE = €1,470.30**, **R² = 0.9027** en **19.62s**, confirmando que **boosting >> regresión lineal**.
  - `XGBoost` obtiene **RMSE = €1,581.94**, **R² = 0.8874** en solo **1.26s**, el más rápido pero con menor precisión.

- **RandomForest vs LightGBM:** Aunque RandomForest logra un R² de 0.896, LightGBM lo supera con 0.903 y es 2.5x más rápido.

### Modelo recomendado

**LGBMR (LightGBM con early stopping)** es el modelo óptimo para producción:
- **Mayor precisión**: RMSE de €1,470.64, el error promedio de predicción es de solo €1,470 por auto.
- **R² = 0.9027**: explica el 90% de la variabilidad del precio.
- **Eficiente**: 13.70s de entrenamiento (27% más rápido que LGB nativo por el early stopping).
- **Robusto**: El early stopping previene sobreajuste automáticamente.

### Recomendaciones para producción
- Guardar el modelo entrenado en caché (con `joblib` o `pickle`) para servir predicciones sin reentrenar.
- Si la velocidad de entrenamiento es crítica (reentrenamiento frecuente), considerar **XGBoost** (1.26s) con leve sacrificio en precisión.
- Reentrenar el modelo semanalmente con datos nuevos y guardar en caché para predicciones en tiempo real.

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos